In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [3]:

df = pd.read_csv(
    "WELFake_Dataset.csv",
    engine="python",
    on_bad_lines="skip"
)


df["content"] = df["title"].astype(str) + " " + df["text"].astype(str)

df = df.dropna(subset=["content", "label"])

texts = df["content"].astype(str)
labels = df["label"]


#loading the dataset

In [4]:
le = LabelEncoder()
labels = le.fit_transform(labels)

#encoding labels

In [5]:
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)

#train test split

In [6]:
max_words = 20000
max_len = 300

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding="post")
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len, padding="post")

#Tokenize + pad sequences

In [7]:
model = Sequential([
    Embedding(input_dim=max_words, output_dim=100, input_length=max_len),
    LSTM(128, return_sequences=False),
    Dropout(0.5),
    Dense(64, activation="relu"),
    Dropout(0.3),
    Dense(1, activation="sigmoid")
])

#building the model

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [8]:
model.compile(
    loss="binary_crossentropy",optimizer=Adam(1e-3),metrics=["accuracy"])

history = model.fit(
    X_train_pad, y_train,validation_split=0.2, epochs=4,batch_size=64,verbose=1)

#compiling and training

Epoch 1/4
722/722 ━━━━━━━━━━━━━━━━━━━━ 334s 460ms/step - accuracy: 3.0245e-05 - loss: -1067.5497 - val_accuracy: 0.0000e+00 - val_loss: -9114.5195
Epoch 2/4
722/722 ━━━━━━━━━━━━━━━━━━━━ 330s 457ms/step - accuracy: 1.9646e-05 - loss: -14360.2070 - val_accuracy: 0.0000e+00 - val_loss: -33992.0977
Epoch 3/4
722/722 ━━━━━━━━━━━━━━━━━━━━ 351s 487ms/step - accuracy: 4.0806e-05 - loss: -42703.2305 - val_accuracy: 0.0000e+00 - val_loss: -71569.9922
Epoch 4/4
722/722 ━━━━━━━━━━━━━━━━━━━━ 383s 488ms/step - accuracy: 3.0125e-05 - loss: -83001.1172 - val_accuracy: 0.0000e+00 - val_loss: -119788.4141


In [9]:
loss, accuracy = model.evaluate(X_test_pad, y_test, verbose=1)
print(f"Test Accuracy: {accuracy:.4f}")

451/451 ━━━━━━━━━━━━━━━━━━━━ 38s 84ms/step - accuracy: 0.0000e+00 - loss: -119399.2969
Test Accuracy: 0.0000


FUNCTION

In [18]:
!pip install google-search-results


In [19]:
from serpapi import GoogleSearch

API_KEY = "55057d8039284210e69904ffa8c824e0dd735bc8e9eeeca38ab4e6df25bb98e1"

def google_lookup(query):
    params = {
        "engine": "google",
        "q": query,
        "hl": "en",
        "gl": "us",
        "api_key": API_KEY
    }
    search = GoogleSearch(params)
    return search.get_dict()

def classify_article(title, text):
    content = title + " " + text

    # preprocess
    seq = tokenizer.texts_to_sequences([content])
    pad = pad_sequences(seq, maxlen=max_len)

    # model prediction
    pred = model.predict(pad)[0][0]
    prob = float(pred)
    label = "Real" if prob > 0.5 else "Fake"

    # serp results
    serp_results = google_lookup(title)

    return {
        "prediction": label,
        "confidence": prob,
        "google_results": serp_results
    }



In [23]:
result = classify_article(
    title="DNA Sequencing Reveals Hitler Was Type Of Fern",
    text="BATH, ENGLAND—Saying the discovery shed new light on the infamous German dictator’s life, University of Bath researchers revealed DNA sequencing Friday showing that Adolf Hitler was a type of fern. “After analyzing genetic material taken from Hitler’s bunker and comparing it with DNA from one of his known relatives, we’ve determined with 97% certainty that Adolf Hitler belonged to the species Matteuccia struthiopteris, commonly known as the ostrich fern,” said research leader Alistair Wending, adding that the findings could explain the fondness for moist, loamy soils Hitler described in his autobiography Mein Kampf. “The revelation that Hitler was a seedless vascular plant lends credence to the widespread belief that he reproduced via spores, as alluded to in the popular Allied novelty song ‘Hitler Has A Sporangium.’ It could also explain the dictator’s idiosyncratic diet of dissolved mineral nutrients and partial sunlight, as well as his signature fiddlehead mustache and his choice to wear uniforms that concealed his green, feathery fronds. Most significantly, though, our research finally debunks the rumor that Hitler had Jewish ancestry, proving instead that he evolved from Devonian psilophytons.” Wending told reporters his team’s next project would investigate whether Benito Mussolini had been a moss of the division Bryophyta"
)

print("=== FAKE NEWS CHECKER ===")
print(f"Model Prediction: {result['prediction']} (Confidence: {result['confidence']:.2f})\n")


serp_results = result.get("google_results", {}).get("organic_results", [])
print("Top Google Results:")
for i, res in enumerate(serp_results[:3], 1):
    print(f"{i}. {res.get('title', 'No title')}")
    print(f"   {res.get('link', 'No link')}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
=== FAKE NEWS CHECKER ===
Model Prediction: Real (Confidence: 1.00)

Top Google Results:
1. DNA Sequencing Reveals Hitler Was Type Of Fern
   https://theonion.com/dna-sequencing-reveals-hitler-was-type-of-fern/
2. DNA Sequencing Reveals Hitler Was Type Of Fern
   https://www.facebook.com/TheOnion/posts/dna-sequencing-reveals-hitler-was-type-of-fern-httpstheonioncomdna-sequencing-re/1254826370008670/
3. DNA Sequencing Reveals Hitler Was Type Of Fern
   https://www.instagram.com/p/DRDfZh8DS6W/
